Водопьян А.О. Хабибуллин Р.А. 2019 г. 

# Z - фактор

Источник: Dranchuk P. M. et al. Calculation of Z factors for natural gases using equations of state //Journal of Canadian Petroleum Technology. – 1975. – Т. 14. – №. 03.

Коэффициент сверхсжимаемости учитывает отклонение свойств реального газа от идеального.

Dranchuk P. M. et al. получили уравнение состояния реального газа адаптировав его к экспериментальным данным Stending and Katz

$$Z = 1 + 
(A_1 +\frac{A_2}{T_r} +\frac{A_3}{T_{r}^3} +\frac{A_4}{T_{r}^4} +\frac{A_5}{T_{r}^5})\rho_{r} + 
(A_6 +\frac{A_7}{T_{r}^2} +\frac{A_8}{T_{r}^8})\rho_{r}^2 - 
A_9(\frac{A_7}{T_{r}} +\frac{A_6}{T_{r}^2})\rho_{r}^5 - 
A_{10} (1 + A_{11} \rho_{r}^2) \frac{\rho_{r}^2}{T_{r}^3}exp(-A_{11}\rho_{r}{2})$$
где:
$$\rho_r = \frac{Z_c P_r}{Z T_r}$$
причем:
$$Z_c = 0.27$$
коэффициенты:
$$A_1 =0.3265$$
$$A_2 =-1.0700$$
$$A_3 =-0.5339$$
$$A_4 =0.01569$$
$$A_5 =-0.05165$$
$$A_6 =0.5475$$
$$A_7 =-0.7361$$
$$A_8 =0.1844$$
$$A_{9} =0.1056$$
$$A_{10} =0.6134$$
$$A_{11} =0.7210$$
а приведенные давление и температура:
$$P_{r} = \frac{P}{P_c}$$
$$T_{r} = \frac{T}{T_c}$$
где $p_c$ и $T_c$ критические давление и температура соответственно

Импорт необходимых модулей

In [ ]:
import sys
sys.path.append('../')
import uniflocpy.uPVT.PVT_fluids as PVT
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import pylab
import uniflocpy.uPVT.PVT_correlations as PVTcorr
%matplotlib inline
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uTools.plotly_workflow as plotly_workflow

Сравнение экспериментального графика Стендинга-Каци и уравнения Дранчука и Абу-Кассема

In [ ]:
def get_z_curve_StandingKatz(tpr):
    """
    Функция позволяет считать данные из нужного файла в зависимости от входного tpr и построить график
    Допустимые значения tpr = 1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.2, 2.4, 2.6, 2.8, 3
    :param tpr: температура приведенная
    :return: данные из графика Cтендинга для этой температуры
    """
    data = pd.read_csv('data\Standing-Katz Chart Data\sk_tpr_{}.txt'.format(int(tpr*100)), sep=';')
    ppr = np.array(pd.DataFrame(data)['x'])
    z = np.array(pd.DataFrame(data)['y'])
    return ppr, z


# Сравним расчетный график с графиком Стендинга
tpr = 1.05
ppr, z = get_z_curve_StandingKatz(tpr)
z_calc = []
pogr = []
i = 0
for p in ppr:
    z_calc.append(PVTcorr.unf_zfactor_DAK_ppr(p, tpr))
    pogr.append((z[i]-z_calc[i])/z[i] * 100)
    i += 1

pylab.figure(figsize=(15,8))
pylab.subplot(211)
pylab.plot(ppr, z, label='По графикам Стендинга-Катца')
pylab.plot(ppr, z_calc, label='расчетный')
pylab.title('Сравнение графиков для tpr = {}'.format(tpr))
pylab.grid()
pylab.legend()
pylab.figure(figsize=(15,8))
pylab.subplot(212)
pylab.plot(ppr, pogr, label='погрешность в %')
pylab.grid()
pylab.legend()
pylab.show()

График Стендинга-Каца

In [ ]:
# построим все графики Стендинга сразу
tpr = [1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.2, 2.4, 2.6, 2.8, 3]
pylab.figure(figsize=(15,8))

for t in tpr:    
    ppr_standing, z_standing = get_z_curve_StandingKatz(t)
    pylab.plot(ppr_standing, z_standing, label='tpr = {}'.format(t))
    this_df_true = pd.DataFrame({f"tpr SK {t}":z_standing}, index = ppr_standing)
    try:
        true_result_df =true_result_df.join(this_df_true, how='outer')
    except:
        true_result_df = this_df_true.copy()
    
    this_calculated_z = []
    for i in ppr_standing:
        this_calculated_z.append(PVTcorr.unf_zfactor_DAK_ppr(i, t))
    this_df = pd.DataFrame({f"tpr {t}":this_calculated_z}, index = ppr_standing)
    try:
        result_df = result_df.join(this_df, how = 'outer')
    except:
        result_df = this_df.copy()
        
        
pylab.grid()
pylab.title('Графики Стендинга-Катца для различных значений tpr')
pylab.legend()
pylab.xlabel('ppr')
pylab.ylabel('z')
pylab.show()

In [ ]:
df_to_compare = result_df.join(true_result_df, how = 'outer')

In [ ]:
traces = plotly_workflow.create_traces_list_for_all_columms(df_to_compare, chosen_mode = 'lines+markers')

In [ ]:
plotly_workflow.plot_func(traces, 'Коэффициент сверхcжимаемости и графики Стендинга-Катца', 'checl.html')